# P-Tuning实战

#### Step1. 导入相关包

In [ ]:
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq,TrainingArguments,Trainer

#### Step2. 加载数据集

In [ ]:
ds = Dataset.load_from_disk("./alpace_data_zh/")
ds

In [ ]:
ds[0]

#### Step3. 数据集预处理

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("Langboat/bloom-38m-zh")
tokenizer

In [ ]:
def process_func(example):
    MAX_LENGTH = 256
    input_ids, attention_mask, labels = [], [], []
    instruction = tokenizer("\n".join(["Human: " + example["instruction"], example["input"]]).strip() + "\n\nAssistant: ")
    response = tokenizer(example["output"] + tokenizer.eos_token)
    input_ids = instruction["input_ids"] + response["input_ids"]
    attention_mask = instruction["attention_mask"] + response["attention_mask"]
    labels = [-100] * len(instruction["input_ids"]) + response["input_ids"]
    if len(input_ids) > MAX_LENGTH:
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    return {"input_ids": input_ids, "attention_mask": attention_mask, "labels": labels}

In [ ]:
tokenized_ds = ds.map(process_func, remove_columns=ds.column_names)

In [ ]:
tokenizer.decode(tokenized_ds[0]["input_ids"])

In [ ]:
tokenizer.decode(list(filter(lambda  x:x != -100, tokenized_ds[1]["labels"])))

#### Step4. 创建模型

In [ ]:
model =  AutoModelForCausalLM.from_pretrained("Langboat/bloom-1b4-zh", low_cpu_mem_usage=True)

In [ ]:
sum(param.numel() for param in model.parameters())

model size: 1.3B
model: 1.3G * 4 ~= 5.2G
gradient: 1.3G * 4 ~= 5.2G
optimizer:  1.3G * 4 * 2 ~= 10.4G
sum: 20.8G

## P-tuning

#### PEFT Step1. 配置文件

In [ ]:
from peft import PromptEncoderConfig, get_peft_model,TaskType,PromptEncoderReparameterizationType

config = PromptEncoderConfig(
    task_type= TaskType.CAUSAL_LM,
    num_virtual_tokens = 10,
    encoder_reparameterization_type=PromptEncoderReparameterizationType.LSTM,
    encoder_dropout=0.1,
    encoder_num_layers=5,
    encoder_hidden_size=1024
)

#### PEFT Step2. 创建模型

In [ ]:
model = get_peft_model(model, config)

In [ ]:
model.print_trainable_parameters()

#### Step5. 配置训练参数

In [ ]:
args = TrainingArguments(
    output_dir="./prompttuning",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    logging_steps=10,
    num_train_epochs=1
)

#### Step6. 创建训练器

In [ ]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_ds,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True)
)

#### Step7. 模型训练

In [ ]:
trainer.train()

#### 加载训练好的PEFT模型

In [ ]:
from peft import PeftModel
peft_model = PeftModel.from_pretrained(model=model,model_id="./prompttuning/checkpoint-1000/")

In [ ]:
peft_model = peft_model.cuda()
ipt = tokenizer("Human: {}\n{}".format("考试有哪些技巧", "").strip() + "\n\nAssistant: ", return_tensors="pt").to(peft_model.device)
print(tokenizer.decode(peft_model.generate(**ipt, max_length=128, do_sample=True)[0], skip_special_tokens=True))

#### Step8. 模型推理

In [ ]:
from transformers import pipeline
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0)

In [ ]:
ipt = "Human: {}\n{}".format("考试有哪些技巧", "").strip() + "\n\nAssistant: "
pipe(ipt, max_length=256, do_sample=True)